In [3]:
### Домашнее задание
### Условие: 
дана таблица с колонками (id, name, salary, managerId), 
студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) 
и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

In [1]:
import pandas as pd
from pprint import pprint
import pyspark

In [2]:
# from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, when, max, month

In [3]:
data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary':['70', '80', '60', '90'], 'ManagerId':['3', '4', 'Null', 'Null']}
pd_dataframe = pd.DataFrame(data)
pd_dataframe

,id,Name,Salary,ManagerId
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


In [4]:
spark = SparkSession.builder.appName('HomeWork#2').getOrCreate()

24/04/14 06:39:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [39]:
df = spark.createDataFrame(pd_dataframe)

In [40]:
df.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



In [41]:
df = df.withColumn('Salary', df['Salary'].cast('int'))

In [42]:
df = df.withColumn('ManagerId', df['ManagerId'].cast('int'))

In [43]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- ManagerId: integer (nullable = true)



In [44]:
df_manager_salary = \
    df.filter(df.ManagerId.isNull()).withColumnRenamed('Salary', 'ManagerSalary')\
    ['id', 'ManagerSalary'] \
    .withColumnRenamed('id', 'Mgrid')
df_manager_salary.show()

+-----+-------------+
|Mgrid|ManagerSalary|
+-----+-------------+
|    3|           60|
|    4|           90|
+-----+-------------+



In [45]:
df = df.join(df_manager_salary, df.ManagerId == df_manager_salary.Mgrid, 'left').drop('Mgrid')
df.show()

+---+-----+------+---------+-------------+
| id| Name|Salary|ManagerId|ManagerSalary|
+---+-----+------+---------+-------------+
|  3|  Sam|    60|     null|         null|
|  4|  Max|    90|     null|         null|
|  1|  Joe|    70|        3|           60|
|  2|Henry|    80|        4|           90|
+---+-----+------+---------+-------------+



In [49]:
df.filter((df.ManagerId.isNotNull()) & (df.Salary > df.ManagerSalary)).select('Name').show()

+----+
|Name|
+----+
| Joe|
+----+

